This is the playground to test coneff code. The main implementation of coneff is in coneff.py

In [1]:
import numpy as np
import nibabel as nib

In [2]:
N = 10000
struc_mat = np.random.rand(N,N)
func_mat = np.random.rand(N,N)

In [3]:
def preprocess_BOLD_coneff(func_mat):
    #TODO: implement BOLD preprocessing
    return func_mat

In [4]:
def threshold_struc_mat(struc_mat, threshold):
    return struc_mat * (struc_mat > threshold)

In [5]:
def compute_2ndstep_mat(struc_mat):
    #TODO: implement 1 degree separation matrix
    return struc_mat

In [6]:
def get_binary_mat(mat):
    return mat > 0

In [7]:
def normlaize_mat(mat):
    max_val = np.max(mat)
    mat = mat / max_val
    return mat

In [8]:
def gradient_descrent(error_th, maxiter, brain_par, skip_bold, lr, use_multistep, averaged_bold_seq, M, noconn, M_2nstep = None, gd_pass = 1):
    #TODO: check if multistep, have to input M_2nstep
    iter = 0
    etem = []
    while error_des > error_th and iter < maxiter:
        error_des = 0
        gradient = np.zeros((brain_par, brain_par))
        
        t = averaged_bold_seq.shape[1]

        for jj in range(skip_bold, t - 1):
            if gd_pass == 1:
                error_des += np.linalg.norm(
                    np.dot(M, averaged_bold_seq[jj, :]) - averaged_bold_seq[jj + 1, :]
                )
            elif gd_pass == 2:
                error_des += np.linalg.norm(
                    np.dot(M, averaged_bold_seq[jj, :])
                    + np.dot(M_2nstep, averaged_bold_seq[jj, :])
                    - averaged_bold_seq[jj + 1, :]
                )

        error_des *= 0.5
        etem.append(error_des)

        for jj in range(skip_bold, t - 1):
            gradient += (
                np.dot(M, averaged_bold_seq[jj, :])
                + np.dot(M_2nstep, averaged_bold_seq[jj, :])
                - averaged_bold_seq[jj + 1, :]
            )

        if gd_pass == 1:
            M = M - lr * gradient
            M = M * noconn
        elif gd_pass == 2:
            M_2nstep = M_2nstep - lr * gradient
            M_2nstep = M_2nstep * noconn
            
        iter += 1
        
    return M, M_2nstep, etem #effective connectivity matrix, error over iterations

In [9]:
def structured_G_causality(struc_mat, bold_seq, norm_opt, use_multistep, use_deconv):
    #TODO: check data structure of struc_mat and bold_seq
    
    norm_opt = 1
    brain_par = 264
    maxiter = 5000
    maxiter2 = 5000
    error_th = 0.016
    lr = 0.000005
    lr2 = 0.00001
    skip_bold = 5
    structural_thres = 0.5
    use_multistep = 1

    averaged_bold_seq = preprocess_BOLD_coneff(bold_seq)
     
    structure_direct = threshold_struc_mat(struc_mat, structural_thres)
    structure_2nstep = threshold_struc_mat(compute_2ndstep_mat(struc_mat), structural_thres)

    if norm_opt == 1:
        print("Initial matrix binarized")
        M = get_binary_mat(structure_direct)
        M_2nstep = get_binary_mat(structure_2nstep)
        
    elif norm_opt == 2:
        print("Initial matrix normalized according to its largest value")
        M = normlaize_mat(structure_direct)
        M_2nstep = normlaize_mat(structure_2nstep)
        
    noconn_M = M.astype(bool)
    noconn_M2 = M_2nstep.astype(bool)

    M, M_2nstep ,etem = gradient_descrent(error_th, maxiter, brain_par, skip_bold, lr, use_multistep, averaged_bold_seq, M, noconn_M, M_2nstep, 1)
    if use_multistep:
        M, M_2nstep, etem_2 = gradient_descrent(error_th, maxiter2, brain_par, skip_bold, lr2, use_multistep, averaged_bold_seq, M,noconn_M, M_2nstep, 2)
    
    
    etem = np.array(etem)
    etem = etem[etem != 0]
    if use_multistep:
        etem2 = np.array(etem2)
        etem2 = etem2[etem2 != 0]

    return M